In [1]:
from pyproj import Transformer
import math
import xarray as xr
import numpy as np 
from rasterio.windows import from_bounds

## Example Area: near Wieselburg

Bounding box: [15.13, 48.15, 15.16, 48.16] --> in WGS84 (EPSG: 4326)

CLMS data is in ETRS89 (EPSG: 3035) so we need to transform the bbox or use projected coordinates [in meters] from the beginning

In [2]:
minlon, minlat, maxlon, maxlat = 15.14, 48.15, 15.16, 48.16
year = 2020

In [3]:
transformer = Transformer.from_crs(
    "EPSG:4326",
    "EPSG:3035",
    always_xy=True
)


def to_3035(minlon, minlat, maxlon, maxlat):
    x0, y0 = transformer.transform(minlon, minlat)
    x1, y1 = transformer.transform(maxlon, minlat)
    x2, y2 = transformer.transform(maxlon, maxlat)
    x3, y3 = transformer.transform(minlon, maxlat)

    minx_3035 = int(math.floor(min(x0, x1, x2, x3)))
    maxx_3035 = int(math.ceil (max(x0, x1, x2, x3)))
    miny_3035 = int(math.floor(min(y0, y1, y2, y3)))
    maxy_3035 = int(math.ceil (max(y0, y1, y2, y3)))

    return minx_3035, miny_3035, maxx_3035, maxy_3035


In [4]:
minx, miny, maxx, maxy = to_3035(minlon, minlat, maxlon, maxlat)

print(minx, miny, maxx, maxy)

4703219 2795136 4704780 2796350


## Open the zarr store

In [5]:
zarr_path = "https://data.eodc.eu/collections/CLMS/CLMS.zarr"

In [6]:
coords = xr.open_zarr(
    zarr_path,
    group="VPP",
    zarr_version=3,
)

/tmp/ipykernel_42135/784050294.py:1: FutureWarning: zarr_version is deprecated, use zarr_format
  coords = xr.open_zarr(


## All accessible datasets:

In Group VPP (Vegetation Phenology and Productivity):
* AMPL: Season Amplitude
* EOSD: End-of-season Date
* EOSV: End-of-season Value
* LENGTH: Season Length
* LSLOPE: Slope of the Green-up Period 
* MAXV: Season Maximum Value
* MINV: Season Minimum Value
* QFLAG: Quality Flag
* RSLOPE: Slope of the Green-down Period
* SOSD: Start-of-season Date 2017
* SOSV: Start-of-season Value
* SPROD: Seasonal Productivity
* TPROD: Total Productivity

In Group ST (Seasonal Trajectory):
* PPI: Plant Phenology Index
* QFLAG: Quality Flag

### Select the desired dataset (here: maximum value (maxv)) and the season (SEASON1 or SEASON2)

In [7]:
maxv_ds = xr.open_zarr(
    zarr_path,
    group="VPP/MAXV",
    zarr_version=3,
    chunks="auto",
)

maxv_zarr = maxv_ds["SEASON1"].assign_coords(
    time=("time", coords["time"].values),
    x=("x", coords["x"].values),
    y=("y", coords["y"].values),
)

/tmp/ipykernel_42135/2014248718.py:1: FutureWarning: zarr_version is deprecated, use zarr_format
  maxv_ds = xr.open_zarr(


### Create a subset of the data with the year and the bbox

In [8]:
subset = maxv_zarr.sel(
    time=year,
    x=slice(minx, maxx),
    y=slice(miny, maxy),
)

print(subset.values)

[[1.5174 1.1691 0.624  ... 2.0518 1.9257 1.6891]
 [1.6219 1.3118 0.8105 ... 1.9843 2.1596 1.4281]
 [1.4867 0.9864 1.2827 ... 2.1321 2.2493 1.6366]
 ...
 [1.2905 1.1697 0.9216 ... 1.2869 0.8163 0.9015]
 [1.3031 1.1622 0.9274 ... 1.2263 0.7417 0.9717]
 [2.0798 2.2828 1.2472 ... 1.1851 0.7516 1.0339]]


### You can also use multiple years and create the subset dataarray

In [9]:
years = [2019, 2020, 2021, 2022]

subset2 = maxv_zarr.sel(
    time=years,
    x=slice(minx, maxx),
    y=slice(miny, maxy),
)

print(subset2)

<xarray.DataArray 'SEASON1' (time: 4, y: 121, x: 156)> Size: 604kB
dask.array<getitem, shape=(4, 121, 156), dtype=float64, chunksize=(1, 121, 156), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) int32 16B 2019 2020 2021 2022
  * x        (x) int32 624B 4703225 4703235 4703245 ... 4704755 4704765 4704775
  * y        (y) int32 484B 2795145 2795155 2795165 ... 2796325 2796335 2796345
Attributes:
    TIFFTAG_COPYRIGHT:  Copernicus service information 2021
    input_time_window:  2016-10-01 to 2021-02-29
    TIIFTAG_SOFTWARE:   Timesat : TIMESAT4.1.8
    PhysRange:          0 to 3
    long_name:          Max of season value (value at peak)


### Calculate the mean value over time

In [10]:
mean = subset2.mean(dim="time", skipna=True)
print(mean.values)

[[1.521675 1.082375 0.697575 ... 1.76215  1.714925 1.759125]
 [1.35735  1.56115  0.966175 ... 1.74225  1.669475 1.51545 ]
 [1.49345  1.236925 1.369675 ... 1.74195  1.717825 1.4313  ]
 ...
 [1.985425 2.000575 1.93155  ... 1.9684   1.690725 1.83885 ]
 [1.7704   2.00375  1.9616   ... 1.97855  1.674075 1.863375]
 [1.6147   1.71995  1.710925 ... 1.9888   1.52115  1.882425]]
